# Apêndice A: Leitura e criação das bases de dados do modelo ("Feature Engineering")

In [214]:
import pandas as pd
import openpyxl
import numpy as np
import os
import glob2
import glob
import random
import sklearn
import ntpath
import json
import datetime
from contextlib import nullcontext
import unicodedata
import read_file_exec as rfe

O intuito desse anexo em Jupyter consiste em estudar os arquivos inputs e modelá-los para datasets a serem usados na aplicação de modelos matemáticos para predição de Vendas, para sugestão de estoque mínimo.

Arquivos input:

* Inventario.xlsx

| Coluna  |  Descrição | 
| ---------- | ---------:              |
| Descrição              |Descrição do produto |
| Código                 |Código de registro único de estoque |
| Estoque Atual          |Estoque atual |
| Estoque Reservado      |Estoque resevado de produto |
| Estoque Disponível     |Estoque Disponível do produto|
| Preço                  |Preço do produto baseado na última atualização |
| Categoria              |Categoria de família de produtos |
| Código Extra           |Códigos extras |
| Unidade                |Unidade do produto |
| Fornecedor Principal   |Fornecedor principal do produto |
| Código Ref.            |Código refência do Produto |
| Marca                  |Marca do Produto |
| Estoque Min.           |Estoque mínimo calculado do sistema |
| Estoque Max.           |Estoque máximo calculado do sistema |
| Abaixo Min.            |Abaixo do mínimo |
| Repor                  |Quantidade a ser reposta |
| Prêmio ?               |Se há alguma bonificação |
| Pontos Nec.            |Campo indefinido |
| Tributação             |Dados tributários |
| NCM                    |Código  |
| Kit / Combo            |Se é parte de kit ou combo |
| EAN / GTIN             |registro nacional do produto eletrônico (Código de barras) |
| Exportar para balança  |Campo indefinido |


----------------------------
* Transações_vendas.xlsx


| Coluna  |  Descrição | 
| ---------- | ----------:
|Ent                 |Campo indefinido          |
|Tipo                |Tipo de transação                 |
|Número              |número da transação                 |
|Data                |data da transação                 |
|Hora                |hora da transação                 |
|Cliente / Fornecedor|Cliente ou fornecedor                 |
|Itens               |Itens vendidos                 |
|Total Final         |Total das transação                 |
|Observações         |Observação da transação                 |
|Funcionário         |Funcionário responsável                 |
|Vendedor            |Vendedor                  |
|Subtotal            |Subtotal da transação                 |
|Desconto            |Desconto da transação                 |
|Outras Despesas     |Outras despesas                 |
|Frete               |Frete                 |
|Valor Pago          |Valor pago pelo cliente                 |
|Meio Pagto          |Meio de pagamento usado                 |
|Crédito Usado       |Crédito utilizado                 |
|Debitado            |Valor debitado                 |
|Troco               |Troco devolvido                 |
|Creditado           |Valor creditado                 |
|Documento           |Nota da transação                 |
|Chave Doc. Fiscal   | Chave do documento fiscal                |
-----------------------------------------------------

* Transações_por_produto.xlsx

| Coluna  |  Descrição | 
| ---------- | ---------: 
|NF                   |Nota fiscal |
|Ent                  |Campo indefinido |
|Tipo                 |Tipo de transação |
|Número               |Número da transação |
|Data                 |Data da transação |
|Hora                 |Hora da transação |
|Cliente / Fornecedor |Cliente ou Fornecedor |
|Produto              |Produto  |
|Quant.               |Transação em quantidades |
|Vl. Unitário         |Valor unitário |
|Sub-Total            |Sub total da transação |
|Desconto             |Desconto aplicado |
|Outras Despesas      |Outras despesas |
|Total                |Total da transação |
|Funcionário          |Funcionário responsável |
|Valor Comissão       |Valor da comissão |
|Comissão %           |Porcentagem da comissão |
|Calc.Comissão        |Cálculo da comissão |

In [215]:
#Função para listar campos nulos de um dataframe e remover as colunas puramente nulas


def list_columns_and_remove_nullcontext(df):
    null_columns= []
    for column in df.columns:
        if df[column].isnull().all():
            null_columns = null_columns + [column]
    print(null_columns)
    df.drop(null_columns, axis=1, inplace=True)

In [216]:

def normalize_string(string):
        return ''.join(char for char in unicodedata.normalize('NFD', string) if unicodedata.category(char) != 'Mn')
    
def lower_string(string):
        return string.lower()

In [217]:
## Função para achar inconsistências em um dataframe
nat_as_integer = np.datetime64('NaT').view('i8')

def isnat(your_datetime):
    dtype_string = str(your_datetime.dtype)
    if 'datetime64[ns]' in dtype_string or 'timestamp64' in dtype_string:
        return your_datetime.view('i8') == nat_as_integer
    return False  # it can't be a NaT if it's not a dateime

In [218]:
def prepare_columns(df):   
    df.columns = [label.replace(' ', '_') for label in df.columns]
    df.columns = [label.replace('/', '_') for label in df.columns]
    df.columns = [label.replace('-', '_') for label in df.columns]
    df.columns = [label.replace('(', '_') for label in df.columns]
    df.columns = [label.replace(')', '_') for label in df.columns]
    df.columns = [label.replace('.', '_') for label in df.columns]
    df.columns = [label.replace(',', '_') for label in df.columns]
    df.columns = [label.replace(';', '_') for label in df.columns]
    df.columns = [label.replace(':', '_') for label in df.columns]
    df.columns = [label.replace('?', '_') for label in df.columns]
    df.columns = [label.replace('!', '_') for label in df.columns]
    df.columns = [label.replace('#', '_') for label in df.columns]
    df.columns = [label.replace('$', '_') for label in df.columns]
    df.columns = [label.replace('%', '_') for label in df.columns]
    df.columns = [label.replace('^', '_') for label in df.columns]
    df.columns = [label.replace('&', '_') for label in df.columns]
    df.columns = [label.replace('*', '_') for label in df.columns]
    df.columns = [label.replace('(', '_') for label in df.columns]
    df.columns = [label.replace(')', '_') for label in df.columns]
    df.columns = [label.replace('+', '_') for label in df.columns]
    df.columns = [label.replace('=', '_') for label in df.columns]
    df.columns = [label.lower() for label in df.columns]
    df.columns = [normalize_string(label) for label in df.columns]

#### Criação dos dataframes do modelo

In [219]:
dataframe_inventario = rfe.read_file('files/Inventario.xlsx').get_df() # Cria dataframe de inventário ## Normaliza os nomes das colunas
prepare_columns(dataframe_inventario) # Normaliza os nomes das colunas
dataframe_transacoes_produto = rfe.read_file('files/Transações_por_produto.xlsx').get_df() #.prepare_columns() # Cria dataframe de transações por produto ## Normaliza os nomes das colunas
prepare_columns(dataframe_transacoes_produto) # Normaliza os nomes das colunas
dataframe_transacoes_vendas = rfe.read_file('files/Transações_vendas.xlsx').get_df()#.prepare_columns() # Cria dataframe de transações por venda ## Normaliza os nomes das colunas
prepare_columns(dataframe_transacoes_vendas) # Normaliza os nomes das colunas
dataframe_ipca = rfe.read_file('files/ipca_modelo.xlsx').get_df()#.prepare_columns() # Cria dataframe de transações por venda ## Normaliza os nomes das colunas
prepare_columns(dataframe_ipca) # Normaliza os nomes das colunas
dataframe_deprec = rfe.read_file('files/NCM.xlsx').get_df()#.prepare_columns() # Cria dataframe de transações por venda ## Normaliza os nomes das colunas
prepare_columns(dataframe_deprec) # Normaliza os nomes das colunas

In [220]:
dataframe_deprec.head(20)

,categoria,ncm,taxa_depreciacao
0,armarinho,8452,10
1,artesanato,6303,20
2,aviamento,8452,10
3,decoracao,6303,20
4,enxoval,6303,20
5,la,6303,20
6,mdf,4415,20
7,outros,*,20
8,papelaria,3926,0
9,roupas,6303,20


In [221]:
dataframe_deprec.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13 entries, 0 to 12
Data columns (total 3 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   categoria         13 non-null     object
 1   ncm               13 non-null     object
 2   taxa_depreciacao  13 non-null     int64 
dtypes: int64(1), object(2)
memory usage: 440.0+ bytes


In [222]:
dataframe_ipca.head(20)

,mes,indice
0,2020-03-01,5348.49
1,2020-04-01,5331.91
2,2020-05-01,5311.65
3,2020-06-01,5325.46
4,2020-07-01,5344.63
5,2020-08-01,5357.46
6,2020-09-01,5391.75
7,2020-10-01,5438.12
8,2020-11-01,5486.52
9,2020-12-01,5560.59


In [223]:
dataframe_ipca.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 29 entries, 0 to 28
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype         
---  ------  --------------  -----         
 0   mes     29 non-null     datetime64[ns]
 1   indice  29 non-null     float64       
dtypes: datetime64[ns](1), float64(1)
memory usage: 592.0 bytes


In [224]:
dataframe_inventario.head()



,descricao,codigo,estoque_atual,estoque_reservado,estoque_disponivel,preco,categoria,codigo_extra,unidade,fornecedor_principal,...,comissao,localizacao,promocao,data_inicial,data_final,preco_promocional,status,kit___combo,ean___gtin,exportar_para_balanca
0,APLIQUE - APM4 -053 NOIVOS,7100692,9.0,0.0,9.0,2.5,PAPELARIA,NaN,NaN,NaN,...,NaN,NaN,NAO,NaT,NaT,NaN,Ativo,Não,7.893798e+12,Não
1,BARROCO MULT 200G -9384 GREENNERY,099810,0.0,0.0,0.0,22.8,Linha,NaN,NaN,NaN,...,NaN,NaN,NAO,NaT,NaT,NaN,Ativo,Não,7.891114e+12,Não
2,ABAJUR MARINHEIRO,124205,1.0,0.0,1.0,38.0,MDF,NaN,NaN,NaN,...,NaN,NaN,NAO,NaT,NaT,NaN,Ativo,Não,NaN,Não
3,ABAJUR MDF DECORADO BEBE URSA,8099512,1.0,0.0,1.0,65.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NAO,NaT,NaT,NaN,Ativo,Não,NaN,Não
4,ABRIDOR DE CASA G,7101086,2.0,0.0,2.0,5.0,ARMARINHO,NaN,NaN,NaN,...,NaN,NaN,NAO,NaT,NaT,NaN,Ativo,Não,NaN,Não


In [225]:
dataframe_inventario.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5261 entries, 0 to 5260
Data columns (total 36 columns):
 #   Column                 Non-Null Count  Dtype         
---  ------                 --------------  -----         
 0   descricao              5261 non-null   object        
 1   codigo                 5247 non-null   object        
 2   estoque_atual          5247 non-null   float64       
 3   estoque_reservado      5245 non-null   float64       
 4   estoque_disponivel     5244 non-null   float64       
 5   preco                  5247 non-null   float64       
 6   categoria              5234 non-null   object        
 7   codigo_extra           0 non-null      float64       
 8   unidade                557 non-null    object        
 9   fornecedor_principal   2 non-null      object        
 10  codigo_ref_            0 non-null      float64       
 11  marca                  3921 non-null   object        
 12  estoque_min_           5247 non-null   float64       
 13  est

In [226]:
dataframe_transacoes_produto.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 47175 entries, 0 to 47174
Data columns (total 18 columns):
 #   Column                Non-Null Count  Dtype         
---  ------                --------------  -----         
 0   nf                    0 non-null      float64       
 1   ent                   0 non-null      float64       
 2   tipo                  37170 non-null  object        
 3   numero                37170 non-null  float64       
 4   data                  37170 non-null  datetime64[ns]
 5   hora                  37170 non-null  datetime64[ns]
 6   cliente___fornecedor  2 non-null      object        
 7   produto               47175 non-null  object        
 8   quant_                47175 non-null  float64       
 9   vl__unitario          47175 non-null  float64       
 10  sub_total             47175 non-null  float64       
 11  desconto              47175 non-null  float64       
 12  outras_despesas       47175 non-null  int64         
 13  total           

In [227]:
dataframe_transacoes_vendas.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 37170 entries, 0 to 37169
Data columns (total 23 columns):
 #   Column                Non-Null Count  Dtype         
---  ------                --------------  -----         
 0   ent                   0 non-null      float64       
 1   tipo                  37170 non-null  object        
 2   numero                37170 non-null  int64         
 3   data                  37170 non-null  datetime64[ns]
 4   hora                  37170 non-null  datetime64[ns]
 5   cliente___fornecedor  3 non-null      object        
 6   itens                 37170 non-null  object        
 7   total_final           37170 non-null  float64       
 8   observacoes           32 non-null     object        
 9   funcionario           37033 non-null  object        
 10  vendedor              9479 non-null   object        
 11  subtotal              9888 non-null   object        
 12  desconto              37170 non-null  float64       
 13  outras_despesas 

#### Checando o dataframe de transação

In [228]:
dataframe_transacoes_produto.head(15)

,nf,ent,tipo,numero,data,hora,cliente___fornecedor,produto,quant_,vl__unitario,sub_total,desconto,outras_despesas,total,funcionario,valor_comissao,comissao__,calc_comissao
0,NaN,NaN,Venda,38253.0,2022-07-30 11:45:35.193,2022-07-30 11:45:35.193,NaN,TECIDO TRICOLINE DIVERSOS,-0.25,27.90,6.97,0.00,0,6.97,Administrador,0,0,Sob.Total
1,NaN,NaN,Venda,38252.0,2022-07-30 09:45:20.596,2022-07-30 09:45:20.596,NaN,FITA CETIM PROGRESSO 0 -209 VERMELHO,-100.00,0.35,35.00,7.00,0,28.00,Administrador,0,0,Sob.Total
2,NaN,NaN,Venda,38251.0,2022-07-30 09:36:37.140,2022-07-30 09:36:37.140,NaN,"FITA MÉTRICA NYBC 1,5m",-1.00,5.80,5.80,0.00,0,5.80,Administrador,0,0,Sob.Total
3,NaN,NaN,NaN,NaN,NaT,NaT,NaN,COSTURA DRIMA - Branca,-1.00,3.00,3.00,0.00,0,3.00,Administrador,0,0,Sob.Total
4,NaN,NaN,NaN,NaN,NaT,NaT,NaN,COSTURA DRIMA - 5605,-1.00,3.00,3.00,0.00,0,3.00,Administrador,0,0,Sob.Total
5,NaN,NaN,Venda,38248.0,2022-07-29 17:29:13.607,2022-07-29 17:29:13.607,NaN,MINIATURA LASER CADEIRA,-2.00,4.80,9.60,0.00,0,9.60,Administrador,0,0,Sob.Total
6,NaN,NaN,NaN,NaN,NaT,NaT,NaN,COLA TEK BOND 2 20G,-1.00,16.80,16.80,0.00,0,16.80,Administrador,0,0,Sob.Total
7,NaN,NaN,Venda,38247.0,2022-07-29 16:37:15.597,2022-07-29 16:37:15.597,NaN,AGULHA COSTURA -Darning MILWARD 5,-3.00,1.00,3.00,0.00,0,3.00,Administrador,0,0,Sob.Total
8,NaN,NaN,NaN,NaN,NaT,NaT,NaN,AGULHA COSTURA -Darning MILWARD 3,-2.00,1.00,2.00,0.00,0,2.00,Administrador,0,0,Sob.Total
9,NaN,NaN,NaN,NaN,NaT,NaT,NaN,AGULHA COSTURA -Darning N 1,-2.00,1.25,2.50,0.00,0,2.50,Administrador,0,0,Sob.Total


Como podemos ver, neste caso tivemos o arquivo agrupado no excel. O mesmo agrupamento não foi mantido na leitura, assim sendo necessário além da remoção de colunas nulas, uma função em que replique o último valor por cada tranação para que complete os dados que competem à mesma transação.

#### Checando o dataframe de inventário

In [229]:
dataframe_inventario.head()

,descricao,codigo,estoque_atual,estoque_reservado,estoque_disponivel,preco,categoria,codigo_extra,unidade,fornecedor_principal,...,comissao,localizacao,promocao,data_inicial,data_final,preco_promocional,status,kit___combo,ean___gtin,exportar_para_balanca
0,APLIQUE - APM4 -053 NOIVOS,7100692,9.0,0.0,9.0,2.5,PAPELARIA,NaN,NaN,NaN,...,NaN,NaN,NAO,NaT,NaT,NaN,Ativo,Não,7.893798e+12,Não
1,BARROCO MULT 200G -9384 GREENNERY,099810,0.0,0.0,0.0,22.8,Linha,NaN,NaN,NaN,...,NaN,NaN,NAO,NaT,NaT,NaN,Ativo,Não,7.891114e+12,Não
2,ABAJUR MARINHEIRO,124205,1.0,0.0,1.0,38.0,MDF,NaN,NaN,NaN,...,NaN,NaN,NAO,NaT,NaT,NaN,Ativo,Não,NaN,Não
3,ABAJUR MDF DECORADO BEBE URSA,8099512,1.0,0.0,1.0,65.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NAO,NaT,NaT,NaN,Ativo,Não,NaN,Não
4,ABRIDOR DE CASA G,7101086,2.0,0.0,2.0,5.0,ARMARINHO,NaN,NaN,NaN,...,NaN,NaN,NAO,NaT,NaT,NaN,Ativo,Não,NaN,Não


Para os arquivos de inventário, é possível remover colunas puramente nulas, pois também, como no dataframe anterior, não agregarão ao modelo.

#### Limpeza e adequação dos dados: 


### Dataframe de Transações por produto

Neste caso, é interessante o uso do método de beforefill do pandas par que o registro posterior receba o valor do anterior, assim ajustando os agrupamentos do arquivo xlsx.

In [230]:
dataframe_transacoes_produto['tipo'] = dataframe_transacoes_produto['tipo'].fillna(method='ffill')
dataframe_transacoes_produto['numero'] = dataframe_transacoes_produto['numero'].fillna(method='ffill')
dataframe_transacoes_produto['data'] = dataframe_transacoes_produto['data'].fillna(method='ffill')
dataframe_transacoes_produto['hora'] = dataframe_transacoes_produto['hora'].fillna(method='ffill')

In [231]:
dataframe_transacoes_produto_tratado = dataframe_transacoes_produto
dataframe_transacoes_produto_tratado.head()

,nf,ent,tipo,numero,data,hora,cliente___fornecedor,produto,quant_,vl__unitario,sub_total,desconto,outras_despesas,total,funcionario,valor_comissao,comissao__,calc_comissao
0,NaN,NaN,Venda,38253.0,2022-07-30 11:45:35.193,2022-07-30 11:45:35.193,NaN,TECIDO TRICOLINE DIVERSOS,-0.25,27.90,6.97,0.0,0,6.97,Administrador,0,0,Sob.Total
1,NaN,NaN,Venda,38252.0,2022-07-30 09:45:20.596,2022-07-30 09:45:20.596,NaN,FITA CETIM PROGRESSO 0 -209 VERMELHO,-100.00,0.35,35.00,7.0,0,28.00,Administrador,0,0,Sob.Total
2,NaN,NaN,Venda,38251.0,2022-07-30 09:36:37.140,2022-07-30 09:36:37.140,NaN,"FITA MÉTRICA NYBC 1,5m",-1.00,5.80,5.80,0.0,0,5.80,Administrador,0,0,Sob.Total
3,NaN,NaN,Venda,38251.0,2022-07-30 09:36:37.140,2022-07-30 09:36:37.140,NaN,COSTURA DRIMA - Branca,-1.00,3.00,3.00,0.0,0,3.00,Administrador,0,0,Sob.Total
4,NaN,NaN,Venda,38251.0,2022-07-30 09:36:37.140,2022-07-30 09:36:37.140,NaN,COSTURA DRIMA - 5605,-1.00,3.00,3.00,0.0,0,3.00,Administrador,0,0,Sob.Total


Neste passo para uma boa análise exploratória dos dados, é interessante remover colunas desinteressantes para o modelo. A função criada no início do modelo pode ser usada para a remoção de campos nulos de cada dataframe.

In [232]:
list_columns_and_remove_nullcontext(dataframe_transacoes_produto_tratado)
dataframe_transacoes_produto_tratado.head()


['nf', 'ent']


,tipo,numero,data,hora,cliente___fornecedor,produto,quant_,vl__unitario,sub_total,desconto,outras_despesas,total,funcionario,valor_comissao,comissao__,calc_comissao
0,Venda,38253.0,2022-07-30 11:45:35.193,2022-07-30 11:45:35.193,NaN,TECIDO TRICOLINE DIVERSOS,-0.25,27.90,6.97,0.0,0,6.97,Administrador,0,0,Sob.Total
1,Venda,38252.0,2022-07-30 09:45:20.596,2022-07-30 09:45:20.596,NaN,FITA CETIM PROGRESSO 0 -209 VERMELHO,-100.00,0.35,35.00,7.0,0,28.00,Administrador,0,0,Sob.Total
2,Venda,38251.0,2022-07-30 09:36:37.140,2022-07-30 09:36:37.140,NaN,"FITA MÉTRICA NYBC 1,5m",-1.00,5.80,5.80,0.0,0,5.80,Administrador,0,0,Sob.Total
3,Venda,38251.0,2022-07-30 09:36:37.140,2022-07-30 09:36:37.140,NaN,COSTURA DRIMA - Branca,-1.00,3.00,3.00,0.0,0,3.00,Administrador,0,0,Sob.Total
4,Venda,38251.0,2022-07-30 09:36:37.140,2022-07-30 09:36:37.140,NaN,COSTURA DRIMA - 5605,-1.00,3.00,3.00,0.0,0,3.00,Administrador,0,0,Sob.Total


In [233]:
dataframe_transacoes_produto_tratado.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 47175 entries, 0 to 47174
Data columns (total 16 columns):
 #   Column                Non-Null Count  Dtype         
---  ------                --------------  -----         
 0   tipo                  47175 non-null  object        
 1   numero                47175 non-null  float64       
 2   data                  47175 non-null  datetime64[ns]
 3   hora                  47175 non-null  datetime64[ns]
 4   cliente___fornecedor  2 non-null      object        
 5   produto               47175 non-null  object        
 6   quant_                47175 non-null  float64       
 7   vl__unitario          47175 non-null  float64       
 8   sub_total             47175 non-null  float64       
 9   desconto              47175 non-null  float64       
 10  outras_despesas       47175 non-null  int64         
 11  total                 47175 non-null  float64       
 12  funcionario           46952 non-null  object        
 13  valor_comissao  

In [234]:

dataframe_transacoes_produto_tratado.isnull().sum()


tipo                        0
numero                      0
data                        0
hora                        0
cliente___fornecedor    47173
produto                     0
quant_                      0
vl__unitario                0
sub_total                   0
desconto                    0
outras_despesas             0
total                       0
funcionario               223
valor_comissao              0
comissao__                  0
calc_comissao           27285
dtype: int64

In [235]:
dataframe_transacoes_produto_tratado.describe().transpose()

,count,mean,std,min,25%,50%,75%,max
numero,47175.0,20458.837880,10843.953093,1.00,11753.5,21035.0,29810.000,38253.0
quant_,47175.0,2.250551,125.751798,-10092.06,-1.0,-1.0,0.000,11988.0
vl__unitario,47175.0,6.850450,17.851295,0.00,0.0,2.5,9.375,1174.0
sub_total,47175.0,5.838035,14.485489,0.00,0.0,0.0,6.000,700.0
desconto,47175.0,0.159503,1.440226,0.00,0.0,0.0,0.000,98.0
outras_despesas,47175.0,0.000000,0.000000,0.00,0.0,0.0,0.000,0.0
total,47175.0,5.678532,13.948205,0.00,0.0,0.0,6.000,700.0
valor_comissao,47175.0,0.000000,0.000000,0.00,0.0,0.0,0.000,0.0
comissao__,47175.0,0.000000,0.000000,0.00,0.0,0.0,0.000,0.0


Como há vários valores nules no campo de clientes/fornecedores, além de metade das transações de cáculo de comissão serem nulas, removeremos os dados destes campos citados mais os dados de funcionário, pois não agregarão ao modelo.

In [236]:
dataframe_transacoes_produto_tratado = dataframe_transacoes_produto_tratado.drop(columns=['cliente___fornecedor','funcionario','calc_comissao','valor_comissao','comissao__'])
dataframe_transacoes_produto_tratado.head()

,tipo,numero,data,hora,produto,quant_,vl__unitario,sub_total,desconto,outras_despesas,total
0,Venda,38253.0,2022-07-30 11:45:35.193,2022-07-30 11:45:35.193,TECIDO TRICOLINE DIVERSOS,-0.25,27.90,6.97,0.0,0,6.97
1,Venda,38252.0,2022-07-30 09:45:20.596,2022-07-30 09:45:20.596,FITA CETIM PROGRESSO 0 -209 VERMELHO,-100.00,0.35,35.00,7.0,0,28.00
2,Venda,38251.0,2022-07-30 09:36:37.140,2022-07-30 09:36:37.140,"FITA MÉTRICA NYBC 1,5m",-1.00,5.80,5.80,0.0,0,5.80
3,Venda,38251.0,2022-07-30 09:36:37.140,2022-07-30 09:36:37.140,COSTURA DRIMA - Branca,-1.00,3.00,3.00,0.0,0,3.00
4,Venda,38251.0,2022-07-30 09:36:37.140,2022-07-30 09:36:37.140,COSTURA DRIMA - 5605,-1.00,3.00,3.00,0.0,0,3.00


Contabilizando valores nulos em cada coluna do dataframe de transações por produto:


### Dataframe de Inventário

O mesmo tratamento de remoção de colunas puramente nulas será feito para o dataframe de inventário.

In [237]:
dataframe_inventario_tratado = dataframe_inventario
list_columns_and_remove_nullcontext(dataframe_inventario_tratado)
dataframe_inventario_tratado.head()

['codigo_extra', 'codigo_ref_', 'pontos_nec_', 'tributacao', 'ncm', 'cest', 'peso_liq_', 'peso_bruto', 'comissao', 'localizacao']


,descricao,codigo,estoque_atual,estoque_reservado,estoque_disponivel,preco,categoria,unidade,fornecedor_principal,marca,...,alterado_em,alterado_por,promocao,data_inicial,data_final,preco_promocional,status,kit___combo,ean___gtin,exportar_para_balanca
0,APLIQUE - APM4 -053 NOIVOS,7100692,9.0,0.0,9.0,2.5,PAPELARIA,NaN,NaN,LITOARTE,...,2021-04-13 15:15:27.708,admin,NAO,NaT,NaT,NaN,Ativo,Não,7.893798e+12,Não
1,BARROCO MULT 200G -9384 GREENNERY,099810,0.0,0.0,0.0,22.8,Linha,NaN,NaN,Círculo,...,2022-05-07 07:56:56.014,admin,NAO,NaT,NaT,NaN,Ativo,Não,7.891114e+12,Não
2,ABAJUR MARINHEIRO,124205,1.0,0.0,1.0,38.0,MDF,NaN,NaN,NaN,...,NaT,NaN,NAO,NaT,NaT,NaN,Ativo,Não,NaN,Não
3,ABAJUR MDF DECORADO BEBE URSA,8099512,1.0,0.0,1.0,65.0,NaN,NaN,NaN,NaN,...,2022-02-08 09:41:08.406,admin,NAO,NaT,NaT,NaN,Ativo,Não,NaN,Não
4,ABRIDOR DE CASA G,7101086,2.0,0.0,2.0,5.0,ARMARINHO,NaN,NaN,NaN,...,2021-10-11 09:35:33.641,admin,NAO,NaT,NaT,NaN,Ativo,Não,NaN,Não


In [238]:
dataframe_inventario_tratado.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5261 entries, 0 to 5260
Data columns (total 26 columns):
 #   Column                 Non-Null Count  Dtype         
---  ------                 --------------  -----         
 0   descricao              5261 non-null   object        
 1   codigo                 5247 non-null   object        
 2   estoque_atual          5247 non-null   float64       
 3   estoque_reservado      5245 non-null   float64       
 4   estoque_disponivel     5244 non-null   float64       
 5   preco                  5247 non-null   float64       
 6   categoria              5234 non-null   object        
 7   unidade                557 non-null    object        
 8   fornecedor_principal   2 non-null      object        
 9   marca                  3921 non-null   object        
 10  estoque_min_           5247 non-null   float64       
 11  estoque_max_           5247 non-null   float64       
 12  abaixo_min_            5247 non-null   object        
 13  rep

In [239]:
dataframe_inventario_tratado.describe().transpose()

,count,mean,std,min,25%,50%,75%,max
estoque_atual,5247.0,1.745982e+01,1.462484e+02,-28.00,1.000000e+00,4.000000e+00,9.000000e+00,5.443000e+03
estoque_reservado,5245.0,0.000000e+00,0.000000e+00,0.00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
estoque_disponivel,5244.0,1.746981e+01,1.462896e+02,-28.00,1.000000e+00,4.000000e+00,9.000000e+00,5.443000e+03
preco,5247.0,1.515103e+01,3.116881e+01,0.06,3.500000e+00,6.800000e+00,1.980000e+01,7.500000e+02
estoque_min_,5247.0,1.905851e-03,1.380526e-01,0.00,0.000000e+00,0.000000e+00,0.000000e+00,1.000000e+01
estoque_max_,5247.0,7.623404e-03,5.522102e-01,0.00,0.000000e+00,0.000000e+00,0.000000e+00,4.000000e+01
repor,1.0,3.850000e+01,NaN,38.50,3.850000e+01,3.850000e+01,3.850000e+01,3.850000e+01
preco_promocional,2.0,1.600000e+01,0.000000e+00,16.00,1.600000e+01,1.600000e+01,1.600000e+01,1.600000e+01
ean___gtin,3630.0,7.840955e+12,2.283233e+12,3410394.00,7.891020e+12,7.891114e+12,7.893798e+12,6.789939e+13


In [240]:
dataframe_inventario_tratado[dataframe_inventario_tratado['categoria'].isnull()]

,descricao,codigo,estoque_atual,estoque_reservado,estoque_disponivel,preco,categoria,unidade,fornecedor_principal,marca,...,alterado_em,alterado_por,promocao,data_inicial,data_final,preco_promocional,status,kit___combo,ean___gtin,exportar_para_balanca
3,ABAJUR MDF DECORADO BEBE URSA,8099512,1.0,0.0,1.0,65.0,NaN,NaN,NaN,NaN,...,2022-02-08 09:41:08.406,admin,NAO,NaT,NaT,NaN,Ativo,Não,NaN,Não
7,PROMOÇÃO,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaT,NaN,NaN,NaT,NaT,NaN,NaN,NaN,NaN,NaN
9,PROMOÇÃO,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaT,NaN,NaN,NaT,NaT,NaN,NaN,NaN,NaN,NaN
11,PROMOÇÃO,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaT,NaN,NaN,NaT,NaT,NaN,NaN,NaN,NaN,NaN
13,PROMOÇÃO,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaT,NaN,NaN,NaT,NaT,NaN,NaN,NaN,NaN,NaN
403,ARGOLA CHAVEIRO SEM CORRENTE,8099803,-14.0,0.0,-14.0,0.3,NaN,NaN,NaN,NaN,...,2022-05-11 12:16:01.547,admin,NAO,NaT,NaT,NaN,Ativo,Não,NaN,Não
868,BOLSA CROCHÊ ALÇA BARROCO TARTARUGA,8099950,0.0,0.0,0.0,130.0,NaN,NaN,NaN,NaN,...,NaT,NaN,NAO,NaT,NaT,NaN,Ativo,Não,NaN,Não
869,BOLSA CROCHÊ FIO DE MALHA P,8099950,1.0,0.0,1.0,60.0,NaN,NaN,NaN,NaN,...,2022-07-29 11:34:28.238,admin,NAO,NaT,NaT,NaN,Ativo,Não,NaN,Não
894,DE ALGODÃO,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaT,NaN,NaN,NaT,NaT,NaN,NaN,NaN,NaN,NaN
975,LA SEDA,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaT,NaN,NaN,NaT,NaT,NaN,NaN,NaN,NaN,NaN


Neste caso temos alguns registros com código vazio. Estes serão desconsiderados do modelo. Repetiremos a análise de registros com categoria nula após a remoção dos citados para um enriquecimento destes casos já que são poucos.

In [241]:
dataframe_inventario_tratado = dataframe_inventario[dataframe_inventario['codigo'].notnull()]

In [242]:
dataframe_inventario_tratado[dataframe_inventario_tratado['codigo'].isnull()]

,descricao,codigo,estoque_atual,estoque_reservado,estoque_disponivel,preco,categoria,unidade,fornecedor_principal,marca,...,alterado_em,alterado_por,promocao,data_inicial,data_final,preco_promocional,status,kit___combo,ean___gtin,exportar_para_balanca


In [243]:
dataframe_inventario_tratado[dataframe_inventario_tratado['categoria'].isnull()]

,descricao,codigo,estoque_atual,estoque_reservado,estoque_disponivel,preco,categoria,unidade,fornecedor_principal,marca,...,alterado_em,alterado_por,promocao,data_inicial,data_final,preco_promocional,status,kit___combo,ean___gtin,exportar_para_balanca
3,ABAJUR MDF DECORADO BEBE URSA,8099512,1.0,0.0,1.0,65.0,NaN,NaN,NaN,NaN,...,2022-02-08 09:41:08.406,admin,NAO,NaT,NaT,NaN,Ativo,Não,NaN,Não
403,ARGOLA CHAVEIRO SEM CORRENTE,8099803,-14.0,0.0,-14.0,0.3,NaN,NaN,NaN,NaN,...,2022-05-11 12:16:01.547,admin,NAO,NaT,NaT,NaN,Ativo,Não,NaN,Não
868,BOLSA CROCHÊ ALÇA BARROCO TARTARUGA,8099950,0.0,0.0,0.0,130.0,NaN,NaN,NaN,NaN,...,NaT,NaN,NAO,NaT,NaT,NaN,Ativo,Não,NaN,Não
869,BOLSA CROCHÊ FIO DE MALHA P,8099950,1.0,0.0,1.0,60.0,NaN,NaN,NaN,NaN,...,2022-07-29 11:34:28.238,admin,NAO,NaT,NaT,NaN,Ativo,Não,NaN,Não
1455,CX PORTA BARALHO,8099862,0.0,0.0,0.0,5.5,NaN,NaN,NaN,NaN,...,NaT,NaN,NAO,NaT,NaT,NaN,Ativo,Não,NaN,Não
1493,DEDEIRA DE BORRACHA,8099737,9.0,0.0,9.0,2.0,NaN,UNDD,NaN,NaN,...,NaT,NaN,NAO,NaT,NaT,NaN,Ativo,Não,NaN,Não
3087,MDF VASO GRANDE,8099768,0.0,0.0,0.0,27.0,NaN,NaN,NaN,NaN,...,NaT,NaN,NAO,NaT,NaT,NaN,Ativo,Não,NaN,Não
3628,MOSQUETÃO CIRCULO PRATA,8099940,36.0,0.0,36.0,3.5,NaN,NaN,NaN,NaN,...,NaT,NaN,NAO,NaT,NaT,NaN,Ativo,Não,7.891113e+12,Não
3977,PASSA ELÁSTICO PINÇA,8099654,6.0,0.0,6.0,2.9,NaN,NaN,NaN,NaN,...,NaT,NaN,NAO,NaT,NaT,NaN,Ativo,Não,NaN,Não
4061,PLACA MDF DECORADA BEBE URSA,8099512,0.0,0.0,0.0,55.0,NaN,NaN,NaN,NaN,...,NaT,NaN,NAO,NaT,NaT,NaN,Ativo,Não,NaN,Não


Tendo apenas 13 casos, é possível atribuir manualmente um valor de categoria como outros para os casos em que o código não foi informado, não sendo algo que enviesará o modelo, pois como visto, temos cerca de 5400 registros. Estes registros comporão não só inventário, mas serão fonte de cruzamento como uma dimensão que enriquecerá as transações com suas famílias de produtos (categoria).

In [244]:
dataframe_inventario_tratado['categoria'] = dataframe_inventario_tratado['categoria'].fillna('Outros')
dataframe_inventario_tratado['estoque_disponivel'] = dataframe_inventario_tratado['estoque_disponivel'].fillna(0)

dataframe_inventario_tratado.head()


C:\Users\otavi\AppData\Local\Temp\ipykernel_3768\954238488.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataframe_inventario_tratado['categoria'] = dataframe_inventario_tratado['categoria'].fillna('Outros')
C:\Users\otavi\AppData\Local\Temp\ipykernel_3768\954238488.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataframe_inventario_tratado['estoque_disponivel'] = dataframe_inventario_tratado['estoque_disponivel'].fillna(0)


,descricao,codigo,estoque_atual,estoque_reservado,estoque_disponivel,preco,categoria,unidade,fornecedor_principal,marca,...,alterado_em,alterado_por,promocao,data_inicial,data_final,preco_promocional,status,kit___combo,ean___gtin,exportar_para_balanca
0,APLIQUE - APM4 -053 NOIVOS,7100692,9.0,0.0,9.0,2.5,PAPELARIA,NaN,NaN,LITOARTE,...,2021-04-13 15:15:27.708,admin,NAO,NaT,NaT,NaN,Ativo,Não,7.893798e+12,Não
1,BARROCO MULT 200G -9384 GREENNERY,099810,0.0,0.0,0.0,22.8,Linha,NaN,NaN,Círculo,...,2022-05-07 07:56:56.014,admin,NAO,NaT,NaT,NaN,Ativo,Não,7.891114e+12,Não
2,ABAJUR MARINHEIRO,124205,1.0,0.0,1.0,38.0,MDF,NaN,NaN,NaN,...,NaT,NaN,NAO,NaT,NaT,NaN,Ativo,Não,NaN,Não
3,ABAJUR MDF DECORADO BEBE URSA,8099512,1.0,0.0,1.0,65.0,Outros,NaN,NaN,NaN,...,2022-02-08 09:41:08.406,admin,NAO,NaT,NaT,NaN,Ativo,Não,NaN,Não
4,ABRIDOR DE CASA G,7101086,2.0,0.0,2.0,5.0,ARMARINHO,NaN,NaN,NaN,...,2021-10-11 09:35:33.641,admin,NAO,NaT,NaT,NaN,Ativo,Não,NaN,Não


In [245]:
dataframe_inventario_tratado.isnull().sum()

descricao                   0
codigo                      0
estoque_atual               0
estoque_reservado           2
estoque_disponivel          0
preco                       0
categoria                   0
unidade                  4690
fornecedor_principal     5245
marca                    1326
estoque_min_                0
estoque_max_                0
abaixo_min_                 0
repor                    5246
premio__                    0
incluido_em                 0
alterado_em              1157
alterado_por             1157
promocao                    0
data_inicial             5245
data_final               5245
preco_promocional        5245
status                      0
kit___combo                 0
ean___gtin               1617
exportar_para_balanca       0
dtype: int64

In [246]:
dataframe_inventario_tratado['categoria'] = dataframe_inventario_tratado['categoria'].apply(lambda x: normalize_string(x)).apply(lambda x: lower_string(x))
dataframe_inventario_tratado['categoria'].value_counts()

C:\Users\otavi\AppData\Local\Temp\ipykernel_3768\3024148263.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataframe_inventario_tratado['categoria'] = dataframe_inventario_tratado['categoria'].apply(lambda x: normalize_string(x)).apply(lambda x: lower_string(x))


linha         1639
armarinho     1197
papelaria     1123
la             492
mdf            415
enxoval        192
artesanato     123
tecido          27
aviamento       23
outros          13
servicos         1
decoracao        1
roupas           1
Name: categoria, dtype: int64

Com os campos agora normalizados pelas funções presentes no na classe read_file normalize_string e lower_string, podemos tratar os dados de inventário para melhor contabilizá-los. Como o intuito desta parte é a melhoria e padronização dos dados, exploraremos os dados assim que tivermos um bom padrão para iniciar as análises.
O padrão deve seguir maneiras em que o dado há redução de ruídos de operação dos códigos de checagem para cruzamento dos dados com os dados de transação.


### Feature Engineering

Explorando as categorias que mais possuem estoque, sendo também as maiores em quantidade de produtos distintos, faremos um rank dos 10 primeiros produtos com maior quantidade de estoque presente.

In [247]:
plot_rank_categoria_quantidade_data = dataframe_inventario_tratado['estoque_disponivel'].groupby(dataframe_inventario_tratado['categoria']).sum().sort_values(ascending=False)
print("Primeira Alteração:{0} ".format(dataframe_inventario_tratado['alterado_em'].min()))
print("Última Alteração:{0} ".format(dataframe_inventario_tratado['alterado_em'].max()))
print(plot_rank_categoria_quantidade_data)


Primeira Alteração:2020-02-05 14:32:03.621000 
Última Alteração:2022-07-30 08:33:39.684000 
categoria
armarinho     58764.445
tecido         9050.890
linha          8899.000
papelaria      7395.190
la             4089.000
mdf            1426.000
aviamento      1124.650
enxoval         519.000
artesanato      234.000
outros          107.500
roupas            2.000
decoracao         0.000
servicos          0.000
Name: estoque_disponivel, dtype: float64


Com essa data de alteração do registro, é possível saber através do último registro presente que nos indica a data de extração do dado, podemos encontrar um fator
no qual representa a diferença entre a data de extração e a data de alteração do registro.
Consideraremos as datas de consideração de data de extração e data de inicio como primeiro registro de log de atividade.


In [248]:
data_extracao = dataframe_transacoes_produto_tratado['data'].max()
data_inicio = dataframe_transacoes_produto_tratado['data'].min()
diff = data_extracao - data_inicio
print("Data do estoque Inicial citado:{0} ".format(data_inicio))
print("Data do estoque presente citado:{0} ".format(data_extracao))
print("Delta:{0}".format(diff))

Data do estoque Inicial citado:2020-02-03 12:53:28.620000 
Data do estoque presente citado:2022-07-30 11:45:35.193000 
Delta:907 days 22:52:06.573000


Consideraremos dados que não possuam dada de inclusão como a data mínima de ínicio presente nos dados de transação utilizando a premissa de que seria um produto presente em estoque antes da implementação do ERP da loja.


Calcularemos uma taxa seguindo a seguinte variação:

* te_inicio_atualizacao = Delta temporal da data de inclusão até a data de extração presente do relatório
* te_atualizacao_presente = Delta temporal da data de alteração até a data de extração presente do relatório
* te_atualizacao_presente = Delta temporal da última atualização do código de inventário até a data de sua criação no sistema

Com isso, temos que:

* te_atualizacao_presente = te_inicio_presente
    fator_atualização = 0, sendo outlier como um novo produto em estoque
* te_atualizacao_presente > te_inicio_presente
    fator_atualização = (te_atualizacao_presente/te_inicio_presente)/100, nos dando um fator de última atualização
* te_inicio_presente <= 120
    fator_atualização = 0, sendo um produto recém incluso no estoque no último semestre

Nota: o "fator_atualizacao", portanto, no dará os itens mais parados em estoque tomando em conta este fator. Quanto mais próximo de 1, menos movimentado é o estoque. Isso não leva em conta seus preços. Esta heurística serve para nos guiar em termos de estoque sem analisar transações e suas tendências apenas para dar uma visão geral de itens que mais são movimentados (para mais ou para menos).

In [249]:

tempo_em_estoque = dataframe_inventario_tratado[['codigo','descricao','categoria','estoque_disponivel','preco','incluido_em','alterado_em']]
tempo_em_estoque['alterado_em'] = pd.to_datetime(tempo_em_estoque['alterado_em'])
 ## Caso não tenha data de inclusão, usa a premissa já existia no estoque
 ## Caso contrário, faz com as datas de inclusão e alteração

tempo_em_estoque_np = tempo_em_estoque['alterado_em'].to_numpy()
true_false_value_index = []
true_false_value_nat = []

for i in tempo_em_estoque_np:
    if isnat(i) == True:
        true_false_value_index.append(tempo_em_estoque['alterado_em'].index)
    true_false_value_nat.append(isnat(i))

        

for i in range(len(true_false_value_nat)):
    if true_false_value_nat[i] == True:
        aux = tempo_em_estoque['incluido_em'][true_false_value_index[0][i]]
        tempo_em_estoque['alterado_em'][true_false_value_index[0][i]] = aux
        
tempo_em_estoque.head(20) 

C:\Users\otavi\AppData\Local\Temp\ipykernel_3768\1343387842.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tempo_em_estoque['alterado_em'] = pd.to_datetime(tempo_em_estoque['alterado_em'])
C:\Users\otavi\AppData\Local\Temp\ipykernel_3768\1343387842.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tempo_em_estoque['alterado_em'][true_false_value_index[0][i]] = aux


,codigo,descricao,categoria,estoque_disponivel,preco,incluido_em,alterado_em
0,7100692,APLIQUE - APM4 -053 NOIVOS,papelaria,9.0,2.5,2020-11-16 17:19:42.000,2021-04-13 15:15:27.708
1,099810,BARROCO MULT 200G -9384 GREENNERY,linha,0.0,22.8,2020-03-09 10:00:50.269,2022-05-07 07:56:56.014
2,124205,ABAJUR MARINHEIRO,mdf,1.0,38.0,2020-08-07 15:58:36.302,2020-08-07 15:58:36.302
3,8099512,ABAJUR MDF DECORADO BEBE URSA,outros,1.0,65.0,2022-02-08 09:40:31.280,2022-02-08 09:41:08.406
4,7101086,ABRIDOR DE CASA G,armarinho,2.0,5.0,2021-01-26 15:42:06.736,2021-10-11 09:35:33.641
5,7100352,ABRIDOR DE CASA P,armarinho,7.0,2.5,2020-10-27 12:58:15.762,2021-10-11 09:35:54.698
6,7100369,ACRIPUFF - AMARELO OURO,papelaria,6.0,2.5,2020-10-27 15:05:06.788,2021-06-25 09:59:20.733
8,7100372,ACRIPUFF - BRANCO,papelaria,2.0,2.5,2020-10-27 15:42:30.541,2021-06-25 09:59:33.484
10,7100370,ACRIPUFF - PRETO,papelaria,4.0,2.5,2020-10-27 15:40:31.781,2021-06-25 09:59:57.732
12,7100371,ACRIPUFF - VERMELHO,papelaria,2.0,2.5,2020-10-27 15:41:18.222,2021-06-25 10:00:10.741


In [250]:
tempo_em_estoque = pd.merge(tempo_em_estoque, dataframe_deprec, on='categoria')
tempo_em_estoque



,codigo,descricao,categoria,estoque_disponivel,preco,incluido_em,alterado_em,ncm,taxa_depreciacao
0,7100692,APLIQUE - APM4 -053 NOIVOS,papelaria,9.0,2.5,2020-11-16 17:19:42.000,2021-04-13 15:15:27.708,3926,0
1,7100369,ACRIPUFF - AMARELO OURO,papelaria,6.0,2.5,2020-10-27 15:05:06.788,2021-06-25 09:59:20.733,3926,0
2,7100372,ACRIPUFF - BRANCO,papelaria,2.0,2.5,2020-10-27 15:42:30.541,2021-06-25 09:59:33.484,3926,0
3,7100370,ACRIPUFF - PRETO,papelaria,4.0,2.5,2020-10-27 15:40:31.781,2021-06-25 09:59:57.732,3926,0
4,7100371,ACRIPUFF - VERMELHO,papelaria,2.0,2.5,2020-10-27 15:41:18.222,2021-06-25 10:00:10.741,3926,0
...,...,...,...,...,...,...,...,...,...
5242,099711,LÃ VINY - 810,la,1.0,19.8,2020-03-06 10:57:20.217,2021-06-18 10:25:14.662,6303,20
5243,7101808,LINHA CHARME - 8001 BRANCA,la,6.0,21.5,2021-11-19 11:09:01.235,2022-05-18 17:32:47.208,6303,20
5244,8099708,LINHA CLÉA 1000 -2314,la,6.0,21.5,2022-04-04 12:17:55.857,2022-05-05 13:15:43.387,6303,20
5245,8099767,Tapete barbante Urso,decoracao,0.0,60.0,2022-05-07 11:57:18.353,2022-05-07 11:57:18.353,6303,20


Aqui criaremos uma maneira de relacionar o tempo de alteração de estoque com a data de número índice de IPCA, para cruzar os dados de valor inicial. Consideraremos para Valor final, o último índice

In [251]:

tempo_em_estoque['ano_mes'] = tempo_em_estoque['alterado_em'].apply(lambda x: x.strftime('%Y-%m')) ## Gerando a chave de ano e mês
dataframe_ipca['ano_mes'] = dataframe_ipca['mes'].apply(lambda x: x.strftime('%Y-%m')) ## Gerando a chave de ano e mês



In [252]:
dataframe_ipca.head()



,mes,indice,ano_mes
0,2020-03-01,5348.49,2020-03
1,2020-04-01,5331.91,2020-04
2,2020-05-01,5311.65,2020-05
3,2020-06-01,5325.46,2020-06
4,2020-07-01,5344.63,2020-07


## Unindo as taxas de valor de correção através do IPCA


In [253]:
tempo_em_estoque = pd.merge(tempo_em_estoque, dataframe_ipca, on='ano_mes')

In [254]:
indice_atual_aplicado =  dataframe_ipca['indice'].loc[dataframe_ipca['mes'] == dataframe_ipca['mes'].max()]
indice_atual_aplicado.to_frame()
indice_atual_aplicado = indice_atual_aplicado.to_frame().iloc[0][0]
indice_atual_aplicado

6411.95

In [255]:
for index, row in tempo_em_estoque.iterrows():
        
        tempo_em_estoque.loc[index,'te_inicio_presente'] = (data_extracao - row['incluido_em']).days #Tempo total de estoque em dias do ínicio de inclusão até última atualização
        tempo_em_estoque.loc[index,'te_atualizacao_presente'] = (data_extracao - row['alterado_em']).days #Tempo total de estoque em dias do ínicio de inclusão até última atualização
        tempo_em_estoque.loc[index,'te_inicio_atualizacao'] = (row['alterado_em'] - row['incluido_em']).days #Tempo total de estoque em dias do ínicio de inclusão até última atualização
        tempo_em_estoque.loc[index,'anos_estoque'] = (data_extracao - row['alterado_em']).days/365.2425 #Tempo total de estoque em anos do ínicio de inclusão até última atualização
        
        if tempo_em_estoque.loc[index,'estoque_disponivel'] == 0:
                  tempo_em_estoque.loc[index,'clf_estoque'] = 'Produto não está em estoque'
                  tempo_em_estoque.loc[index,'fator_atualizacao'] = 1
        elif tempo_em_estoque.loc[index,'te_inicio_presente'] <= 90:
              tempo_em_estoque.loc[index,'clf_estoque'] = 'Produto alterado no último trimestre'
              try:
                      tempo_em_estoque.loc[index,'fator_atualizacao'] = (tempo_em_estoque.loc[index,'te_atualizacao_presente'] / tempo_em_estoque.loc[index,'te_inicio_presente']) - 1
              except ZeroDivisionError:
                        tempo_em_estoque.loc[index,'fator_atualizacao'] == 0 

        elif tempo_em_estoque.loc[index,'te_inicio_presente'] > tempo_em_estoque.loc[index,'te_atualizacao_presente']:
                tempo_em_estoque.loc[index,'clf_estoque'] = 'Produto no estoque'
                tempo_em_estoque.loc[index,'fator_atualizacao'] = (tempo_em_estoque.loc[index,'te_atualizacao_presente']/tempo_em_estoque.loc[index,'te_inicio_presente'])
        else:
                tempo_em_estoque.loc[index,'clf_estoque'] = 'Risco de estoque parado'
                tempo_em_estoque.loc[index,'fator_atualizacao'] = 1  



for index, row in tempo_em_estoque.iterrows():
        if row['anos_estoque'] < 1:
                tempo_em_estoque.loc[index,'depreciacao_unit'] = 0
        else:
                tempo_em_estoque.loc[index,'depreciacao_unit'] = row['preco']*((row['taxa_depreciacao']/100)**row['anos_estoque'])
        tempo_em_estoque.loc[index,'fator_correcao']  = indice_atual_aplicado/row['indice']
        tempo_em_estoque.loc[index,'correcao_unit'] = row['preco']*row['fator_correcao']
        tempo_em_estoque.loc[index,'correcao_deprec'] = (row['preco']*row['fator_correcao']) - (row['preco']*((row['taxa_depreciacao']/100)**row['anos_estoque']))
        tempo_em_estoque.loc[index,'depreciacao'] = tempo_em_estoque.loc[index,'depreciacao_unit']*row['estoque_disponivel']
        tempo_em_estoque.loc[index,'valor_ini'] = row['estoque_disponivel']*row['preco']
        tempo_em_estoque.loc[index,'valor_estoque_deprec'] = (row['estoque_disponivel']*row['preco']*(((row['taxa_depreciacao']/100)**row['anos_estoque'])))
        tempo_em_estoque.loc[index,'valor_estoque_depreciado_corrigido'] = (row['preco']*(row['fator_correcao'])*((row['taxa_depreciacao']/100)**row['anos_estoque'])*row['estoque_disponivel'])


KeyError: 'fator_correcao'

In [ ]:
tempo_em_estoque.head(5).sort_values(by=['fator_atualizacao'], ascending=False)


,codigo,descricao,categoria,estoque_disponivel,preco,incluido_em,alterado_em,ncm,taxa_depreciacao,ano_mes,...,clf_estoque,fator_atualizacao,depreciacao_unit,fator_correcao,correcao_unit,correcao_deprec,depreciacao,valor_ini,valor_estoque_deprec,valor_estoque_depreciado_corrigido
1,099939,MASSA BISCUIT acrilex 90G - AMARELO PELE,papelaria,0.0,4.5,2020-03-12 09:25:38.569,2021-04-13 14:54:44.957,3926,0,2021-04,...,Produto não está em estoque,1.00000,0.0,1.126423,5.068904,5.068904,0.0,0.0,0.0,0.0
2,099935,MASSA BISCUIT acrilex 90G - VERDE MUSGO,papelaria,0.0,4.5,2020-03-12 09:11:30.999,2021-04-13 14:55:19.701,3926,0,2021-04,...,Produto não está em estoque,1.00000,0.0,1.126423,5.068904,5.068904,0.0,0.0,0.0,0.0
0,7100692,APLIQUE - APM4 -053 NOIVOS,papelaria,9.0,2.5,2020-11-16 17:19:42.000,2021-04-13 15:15:27.708,3926,0,2021-04,...,Produto no estoque,0.76129,0.0,1.126423,2.816058,2.816058,0.0,22.5,0.0,0.0
3,099993,PINCEL TIGRE 815 - 04,papelaria,3.0,3.5,2020-03-12 16:23:51.521,2021-04-30 17:10:25.288,3926,0,2021-04,...,Produto no estoque,0.52359,0.0,1.126423,3.942481,3.942481,0.0,10.5,0.0,0.0
4,099996,PINCEL TIGRE 815 - 10,papelaria,4.0,5.0,2020-03-12 16:28:07.014,2021-04-30 17:11:11.816,3926,0,2021-04,...,Produto no estoque,0.52359,0.0,1.126423,5.632116,5.632116,0.0,20.0,0.0,0.0


In [268]:
#bring unique values of codigo and categoria from tempo_em_estoque
#check duplicates 
tempo_em_estoque_uniq = dataframe_inventario_tratado[['codigo','categoria','descricao']]


#check duplicates 
tempo_em_estoque_uniq.duplicated().sum()
tempo_em_estoque_uniq.rename(columns={'descricao': 'produto'}, inplace=True)
tempo_em_estoque_uniq


C:\Users\otavi\AppData\Local\Temp\ipykernel_3768\1226501952.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tempo_em_estoque_uniq.rename(columns={'descricao': 'produto'}, inplace=True)


,codigo,categoria,produto
0,7100692,papelaria,APLIQUE - APM4 -053 NOIVOS
1,099810,linha,BARROCO MULT 200G -9384 GREENNERY
2,124205,mdf,ABAJUR MARINHEIRO
3,8099512,outros,ABAJUR MDF DECORADO BEBE URSA
4,7101086,armarinho,ABRIDOR DE CASA G
...,...,...,...
5256,2000247-025 428,armarinho,Ziper invisivel 25cm rosa
5257,2000247-025 428,armarinho,Ziper invisivel 30cm vermelho
5258,7101529,armarinho,ZIPER JAQUETA JACARÉ - 65CM PRETO
5259,7101530,armarinho,ZIPER JAQUETA JACARÉ - 70CM PRETO


In [274]:
dataframe_transacoes_produto_tratado = dataframe_transacoes_produto_tratado.merge(tempo_em_estoque_uniq, on='produto', how='left')

In [270]:
tempo_em_estoque[tempo_em_estoque['clf_estoque'] == 'Produto alterado no último trimestre'].sort_values(by=['fator_atualizacao'], ascending=False).head(5)


,codigo,descricao,categoria,estoque_disponivel,preco,incluido_em,alterado_em,ncm,taxa_depreciacao,ano_mes,mes,indice,te_inicio_presente,te_atualizacao_presente,te_inicio_atualizacao,anos_estoque,clf_estoque,fator_atualizacao,depreciacao_unit,fator_correcao
2405,8099907,COLA BRANCA MARIPEL 1KG,papelaria,9.0,25.0,2022-07-05 08:48:05.732,2022-07-05 08:48:05.732,3926,0,2022-07,2022-07-01,6411.95,25.0,25.0,0.0,0.068448,Produto alterado no último trimestre,0.0,NaN,NaN
3378,8099821,CORRENTE P/ BIJU,armarinho,2.0,8.0,2022-05-12 13:35:51.630,2022-05-12 13:35:51.630,8452,10,2022-05,2022-05-01,6412.88,78.0,78.0,0.0,0.213557,Produto alterado no último trimestre,0.0,NaN,NaN
3356,8099787,ARAME ALUMINIO OURO,armarinho,1.0,3.9,2022-05-11 09:11:50.473,2022-05-11 09:11:50.473,8452,10,2022-05,2022-05-01,6412.88,80.0,80.0,0.0,0.219033,Produto alterado no último trimestre,0.0,NaN,NaN
3358,8099804,ARGOLA CHAVEIRO FININHA,armarinho,-6.0,0.3,2022-05-11 12:13:34.070,2022-05-11 12:15:25.837,8452,10,2022-05,2022-05-01,6412.88,79.0,79.0,0.0,0.216295,Produto alterado no último trimestre,0.0,NaN,NaN
3360,8099802,ARGOLA CHAVEIRO OURO VELHO,armarinho,17.0,0.5,2022-05-11 12:10:11.301,2022-05-11 12:10:11.301,8452,10,2022-05,2022-05-01,6412.88,79.0,79.0,0.0,0.216295,Produto alterado no último trimestre,0.0,NaN,NaN


In [294]:
dataframe_transacoes_produto_tratado_1 = dataframe_transacoes_produto_tratado
dataframe_transacoes_produto_tratado_1

,tipo,numero,data,hora,produto,quant_,vl__unitario,sub_total,desconto,outras_despesas,total,codigo_x,categoria_x,codigo_y,categoria_y,codigo,categoria
0,Venda,38253.0,2022-07-30 11:45:35.193,2022-07-30 11:45:35.193,TECIDO TRICOLINE DIVERSOS,-0.25,27.90,6.97,0.0,0,6.97,123889,tecido,123889,tecido,123889,tecido
1,Venda,38252.0,2022-07-30 09:45:20.596,2022-07-30 09:45:20.596,FITA CETIM PROGRESSO 0 -209 VERMELHO,-100.00,0.35,35.00,7.0,0,28.00,7099759,armarinho,7099759,armarinho,7099759,armarinho
2,Venda,38251.0,2022-07-30 09:36:37.140,2022-07-30 09:36:37.140,"FITA MÉTRICA NYBC 1,5m",-1.00,5.80,5.80,0.0,0,5.80,099927,armarinho,099927,armarinho,099927,armarinho
3,Venda,38251.0,2022-07-30 09:36:37.140,2022-07-30 09:36:37.140,COSTURA DRIMA - Branca,-1.00,3.00,3.00,0.0,0,3.00,N126100-0000B,linha,N126100-0000B,linha,N126100-0000B,linha
4,Venda,38251.0,2022-07-30 09:36:37.140,2022-07-30 09:36:37.140,COSTURA DRIMA - 5605,-1.00,3.00,3.00,0.0,0,3.00,123557,linha,123557,linha,123557,linha
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49109,Ajuste de Estoque +,5.0,2020-02-03 13:29:52.440,2020-02-03 13:29:52.440,Linha Rayza - Laranja,5.00,0.00,0.00,0.0,0,0.00,98033,linha,98033,linha,98033,linha
49110,Ajuste de Custo,4.0,2020-02-03 13:27:48.967,2020-02-03 13:27:48.967,Linha Rayza - Verde Bandeira,0.00,5.50,0.00,0.0,0,0.00,98043,linha,98043,linha,98043,linha
49111,Ajuste de Estoque +,3.0,2020-02-03 13:27:48.958,2020-02-03 13:27:48.958,Linha Rayza - Verde Bandeira,9.00,0.00,0.00,0.0,0,0.00,98043,linha,98043,linha,98043,linha
49112,Ajuste de Custo,2.0,2020-02-03 12:53:28.688,2020-02-03 12:53:28.688,CLEA 1000 BRANCO,0.00,8.17,0.00,0.0,0,0.00,NaN,NaN,NaN,NaN,NaN,NaN


In [293]:
dataframe_transacoes_produto_tratado_1[dataframe_transacoes_produto_tratado_1['categoria'].isnull()].to_excel('etl_results/dataframe_transacoes_produto_tratado_1.xlsx')


PermissionError: [Errno 13] Permission denied: 'etl_results/dataframe_transacoes_produto_tratado_1.xlsx'

In [ ]:
try:
    tempo_em_estoque.to_excel('etl_results/tempo_em_estoque.xlsx',index=False)
    dataframe_transacoes_produto_tratado.to_excel('etl_results/transac_por_prod.xlsx',index=False)
    print('Extração e transformação concluída Concluída : \n'
          'Arquivos gerados: \n'
          'etl_results/tempo_em_estoque.xlsx \n'
          'etl_results/transac_por_prod.xlsx \n')
except:
    print('Erro ao gerar arquivos')
    

Extração e transformação concluída Concluída : 
Arquivos gerados: 
etl_results/tempo_em_estoque.xlsx 
etl_results/transac_por_prod.xlsx 



In [314]:
from nltk.test.portuguese_en_fixt import setup_module
import nltk

nltk.download('stopwords')
nltk.download('rslp')
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')
nltk.download('maxent_ne_chunker')
nltk.download('words')

from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import RSLPStemmer
from nltk.stem import WordNetLemmatizer
from nltk import pos_tag
from nltk import ne_chunk
from nltk import word_tokenize
from nltk import sent_tokenize
from nltk import RegexpParser
from nltk import Tree
from nltk import FreqDist
from nltk import ConditionalFreqDist
from nltk import ConditionalProbDist
from nltk import MLEProbDist
from nltk import NaiveBayesClassifier
import re


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\otavi\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package rslp to
[nltk_data]     C:\Users\otavi\AppData\Roaming\nltk_data...
[nltk_data]   Package rslp is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\otavi\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\otavi\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\otavi\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package maxent_ne_chunker to
[nltk_data]     C:\Users\otavi\AppData\Roaming\nltk_data...
[nltk_data]   Package maxent_ne_chunker is already up-to

In [349]:
#create a function to remove stopwords
def remove_stopwords(text):
    stop_words = set(stopwords.words('portuguese'))
    word_tokens = word_tokenize(text)
    filtered_sentence = [w for w in word_tokens if not w in stop_words]
    return filtered_sentence
\
#apply the function to the dataframe
#replace special characters and numbers from the text function
def replace_numbers(text):
    text = re.sub(r'[0-9]', '', text)
    return text


In [355]:
dataframe_transacoes_produto_tratado_1['produto_sem_cs'] = dataframe_transacoes_produto_tratado['produto'].apply(lambda x: normalize_string(x)).apply(lambda x: lower_string(x)).apply(lambda x: replace_numbers(x)).apply(lambda x: remove_stopwords(x)).apply(lambda x: ' '.join(x))


In [356]:
dataframe_transacoes_produto_tratado_2 = dataframe_transacoes_produto_tratado_1[dataframe_transacoes_produto_tratado_1['categoria'].isnull()]
dataframe_transacoes_produto_tratado_3 = dataframe_transacoes_produto_tratado_1[dataframe_transacoes_produto_tratado_1['categoria'].notnull()]


In [359]:

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(dataframe_transacoes_produto_tratado_3['produto_sem_cs'], dataframe_transacoes_produto_tratado_3['categoria'], test_size=0.2, random_state=42)

from sklearn.feature_extraction.text import CountVectorizer
count_vect = CountVectorizer()
X_train_counts = count_vect.fit_transform(X_train)
X_train_counts.shape


(37166, 1669)

In [363]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_extraction.text import TfidfTransformer
tfidf_transformer = TfidfTransformer()
X_train_tfidf = tfidf_transformer.fit_transform(X_train_counts)
X_train_tfidf.shape

classify = MultinomialNB().fit(X_train_tfidf, y_train)


In [369]:
#prediction accuracy
from sklearn.metrics import accuracy_score
accuracy_score(y_test, classify.predict(count_vect.transform(X_test)))

0.9777227722772277

In [366]:
dataframe_transacoes_produto_tratado_2['categoria'] = classify.predict(count_vect.transform(dataframe_transacoes_produto_tratado_2['produto_sem_cs']))


C:\Users\otavi\AppData\Local\Temp\ipykernel_3768\2403356323.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataframe_transacoes_produto_tratado_2['categoria'] = classify.predict(count_vect.transform(dataframe_transacoes_produto_tratado_2['produto_sem_cs']))


In [367]:
dataframe_transacoes_produto_tratado_2

,tipo,numero,data,hora,produto,quant_,vl__unitario,sub_total,desconto,outras_despesas,total,codigo_x,categoria_x,codigo_y,categoria_y,codigo,categoria,produto_sem_,produto_sem_stopwords,produto_sem_cs
353,Ajuste de Custo,38041.0,2022-07-20 17:40:01.564,2022-07-20 17:40:01.564,LINHA CLÉA 125 -5899 PANTANAL,0.0,9.25,0.0,0.0,0,0.0,NaN,NaN,NaN,NaN,NaN,linha,"[linha, clea, pantanal]","[LINHA, CLÉA, 125, -5899, PANTANAL]",linha clea - pantanal
354,Ajuste de Estoque +,38040.0,2022-07-20 17:40:01.374,2022-07-20 17:40:01.374,LINHA CLÉA 125 -5899 PANTANAL,10.0,0.00,0.0,0.0,0,0.0,NaN,NaN,NaN,NaN,NaN,linha,"[linha, clea, pantanal]","[LINHA, CLÉA, 125, -5899, PANTANAL]",linha clea - pantanal
2119,Ajuste de Custo,36992.0,2022-06-16 10:35:44.146,2022-06-16 10:35:44.146,"AGULHA CROCHE LULI - 5,5",0.0,3.00,0.0,0.0,0,0.0,NaN,NaN,NaN,NaN,NaN,armarinho,"[agulha, croche, luli, ,]","[AGULHA, CROCHE, LULI, -, 5,5]","agulha croche luli - ,"
2120,Ajuste de Estoque +,36991.0,2022-06-16 10:35:43.968,2022-06-16 10:35:43.968,"AGULHA CROCHE LULI - 5,5",12.0,0.00,0.0,0.0,0,0.0,NaN,NaN,NaN,NaN,NaN,armarinho,"[agulha, croche, luli, ,]","[AGULHA, CROCHE, LULI, -, 5,5]","agulha croche luli - ,"
2509,Venda,36767.0,2022-06-09 10:42:36.640,2022-06-09 10:42:36.640,PAPEL DECOUPAGE-AFQ-032,-1.0,3.00,3.0,0.0,0,3.0,NaN,NaN,NaN,NaN,NaN,papelaria,"[papel, decoupage, afq]","[PAPEL, DECOUPAGE-AFQ-032]",papel decoupage-afq-
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
48850,Ajuste de Custo,264.0,2020-02-03 18:10:34.009,2020-02-03 18:10:34.009,Meada Anchor - 00880,0.0,1.55,0.0,0.0,0,0.0,NaN,NaN,NaN,NaN,NaN,linha,"[meada, anchor]","[Meada, Anchor, -, 00880]",meada anchor -
48856,Ajuste de Custo,258.0,2020-02-03 18:07:31.383,2020-02-03 18:07:31.383,Meada Anchor - 01080,0.0,1.55,0.0,0.0,0,0.0,NaN,NaN,NaN,NaN,NaN,linha,"[meada, anchor]","[Meada, Anchor, -, 01080]",meada anchor -
49043,Ajuste de Custo,71.0,2020-02-03 13:54:58.368,2020-02-03 13:54:58.368,Linha Rayza - Cabaré,0.0,5.50,0.0,0.0,0,0.0,NaN,NaN,NaN,NaN,NaN,linha,"[linha, rayza, cabare]","[Linha, Rayza, -, Cabaré]",linha rayza - cabare
49112,Ajuste de Custo,2.0,2020-02-03 12:53:28.688,2020-02-03 12:53:28.688,CLEA 1000 BRANCO,0.0,8.17,0.0,0.0,0,0.0,NaN,NaN,NaN,NaN,NaN,linha,"[clea, branco]","[CLEA, 1000, BRANCO]",clea branco


In [374]:
dataframe_transacoes_produto_tratado_final = pd.concat([dataframe_transacoes_produto_tratado_2, dataframe_transacoes_produto_tratado_3], ignore_index=True)
dataframe_transacoes_produto_tratado_final = dataframe_transacoes_produto_tratado_final[['tipo','numero','data','hora',	'produto',	'quant_',	'vl__unitario',	'sub_total',	'desconto',	'outras_despesas',	'total','codigo','categoria']]


In [390]:
for i in range(len(dataframe_transacoes_produto_tratado_final)):
    if dataframe_transacoes_produto_tratado_final['codigo'][i].isnull():
        dataframe_transacoes_produto_tratado_final['codigo'][i] = random(1000000000)+ 'Nao_econtrado' 

        

ValueError: too many values to unpack (expected 2)

In [383]:
dataframe_transacoes_produto_tratado_final

,tipo,numero,data,hora,produto,quant_,vl__unitario,sub_total,desconto,outras_despesas,total,codigo,categoria
0,Ajuste de Custo,38041.0,2022-07-20 17:40:01.564,2022-07-20 17:40:01.564,LINHA CLÉA 125 -5899 PANTANAL,0.0,9.25,0.0,0.0,0,0.0,NaN,linha
1,Ajuste de Estoque +,38040.0,2022-07-20 17:40:01.374,2022-07-20 17:40:01.374,LINHA CLÉA 125 -5899 PANTANAL,10.0,0.00,0.0,0.0,0,0.0,NaN,linha
2,Ajuste de Custo,36992.0,2022-06-16 10:35:44.146,2022-06-16 10:35:44.146,"AGULHA CROCHE LULI - 5,5",0.0,3.00,0.0,0.0,0,0.0,NaN,armarinho
3,Ajuste de Estoque +,36991.0,2022-06-16 10:35:43.968,2022-06-16 10:35:43.968,"AGULHA CROCHE LULI - 5,5",12.0,0.00,0.0,0.0,0,0.0,NaN,armarinho
4,Venda,36767.0,2022-06-09 10:42:36.640,2022-06-09 10:42:36.640,PAPEL DECOUPAGE-AFQ-032,-1.0,3.00,3.0,0.0,0,3.0,NaN,papelaria
...,...,...,...,...,...,...,...,...,...,...,...,...,...
49109,Ajuste de Estoque +,7.0,2020-02-03 13:31:23.044,2020-02-03 13:31:23.044,Linha Rayza - Cinza,9.0,0.00,0.0,0.0,0,0.0,98012,linha
49110,Ajuste de Custo,6.0,2020-02-03 13:29:52.455,2020-02-03 13:29:52.455,Linha Rayza - Laranja,0.0,5.50,0.0,0.0,0,0.0,98033,linha
49111,Ajuste de Estoque +,5.0,2020-02-03 13:29:52.440,2020-02-03 13:29:52.440,Linha Rayza - Laranja,5.0,0.00,0.0,0.0,0,0.0,98033,linha
49112,Ajuste de Custo,4.0,2020-02-03 13:27:48.967,2020-02-03 13:27:48.967,Linha Rayza - Verde Bandeira,0.0,5.50,0.0,0.0,0,0.0,98043,linha


In [ ]:
try:
    tempo_em_estoque.to_excel('etl_results/tempo_em_estoque.xlsx',index=False)
    dataframe_transacoes_produto_tratado_final.to_excel('etl_results/transacoes_produto_tratado_final.xlsx',index=False)
    print('Extração e transformação concluída Concluída : \n'
          'Arquivos gerados: \n'
          'etl_results/tempo_em_estoque.xlsx \n'
          'etl_results/transac_por_prod.xlsx \n')
except:
    print('Erro ao gerar arquivos')